# 08 - Expand Opponents

Compute the opponents faced by our 24 tracked teams, list all opponents by frequency, and export the full list + top 200.

In [1]:
import sys
from pathlib import Path

def _find_root():
    cand = Path.cwd()
    for c in [cand, *cand.parents]:
        if (c / "src").exists() and (c / "data").exists():
            return c
    return cand

ROOT = _find_root()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))
print(f"Using ROOT={ROOT}")

from src.dota_data import read_processed_tables, build_team_dictionary
import polars as pl

processed_dir = ROOT / "data" / "processed"
teams_csv = ROOT / "data" / "teams_to_look.csv"
out_dir = ROOT / "data" / "interim"
out_dir.mkdir(parents=True, exist_ok=True)

Using ROOT=/home/ju/Documents/Dev/Dota-Datas


In [2]:
# Load tables and tracked team IDs
tables = read_processed_tables(processed_dir)
matches = tables["matches"]
teams_dict = build_team_dictionary(matches)
tracked = pl.read_csv(teams_csv)
tracked_ids = tracked["TeamID"].to_list()
tracked_names = dict(zip(tracked["TeamID"], tracked["TeamName"]))
print(f"Tracked teams: {len(tracked_ids)}")

Tracked teams: 24


In [3]:
# Compute opponents: for matches where one side is tracked, count the other side
m = matches.select([
    "match_id",
    "radiant_team_id",
    "dire_team_id",
])

# Radiant tracked -> opponent is Dire; Dire tracked -> opponent is Radiant
rad = (
    m.filter(pl.col("radiant_team_id").is_in(tracked_ids))
    .select(pl.col("dire_team_id").alias("opponent_id"))
)
dire = (
    m.filter(pl.col("dire_team_id").is_in(tracked_ids))
    .select(pl.col("radiant_team_id").alias("opponent_id"))
)
opp = pl.concat([rad, dire]).filter(pl.col("opponent_id").is_not_null())
opp_counts = opp.group_by("opponent_id").agg(pl.len().alias("matches_vs_tracked")).sort("matches_vs_tracked", descending=True)
print(f"Opponents found: {opp_counts.height}")

Opponents found: 174


In [4]:
# Attach names from team dictionary
names_lookup = (
    teams_dict.group_by("team_id")
    .agg(pl.col("name").drop_nulls().first().alias("name"))
    .rename({"team_id": "opponent_id"})
)
opp_named = opp_counts.join(names_lookup, on="opponent_id", how="left")

# Add flag if in tracked list
opp_named = opp_named.with_columns(pl.col("opponent_id").is_in(tracked_ids).alias("is_tracked"))
opp_named.head(10)

opponent_id,matches_vs_tracked,name,is_tracked
i64,u32,str,bool
2163,249,"""Team Liquid""",true
8291895,247,"""Tundra Esports""",true
8255888,214,"""BB Team""",true
9247354,211,"""Team Falcons""",true
9467224,192,"""Aurora Gaming""",true
7119388,189,"""Team Spirit""",true
9572001,181,"""PVISION""",true
8261500,165,"""Xtreme Gaming""",true
9640842,161,"""Team Tidebound""",true


In [5]:
# Save full list and top 200
out_all = out_dir / "opponents_all.csv"
out_top = out_dir / "opponents_top200.csv"
opp_named.write_csv(out_all)
opp_named.head(200).write_csv(out_top)
print(f"Saved all opponents to {out_all} and top200 to {out_top}")

Saved all opponents to /home/ju/Documents/Dev/Dota-Datas/data/interim/opponents_all.csv and top200 to /home/ju/Documents/Dev/Dota-Datas/data/interim/opponents_top200.csv


In [ ]:
# Display top 20 opponents by frequency
opp_named.head(20)